In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from src.umapper import umapper

Load data

In [ ]:
data_dir = "../data/raw/"
groupname = "is20016_by4741"

In [ ]:
filepath = data_dir + groupname
timeseries_filepath = filepath + "_timeseries.csv"
labels_filepath = filepath + "_labels.csv"

timeseries_df = pd.read_csv(timeseries_filepath, index_col=[0,1,2])
labels_df = pd.read_csv(labels_filepath, index_col=[0,1,2])

In [ ]:
timeseries_df

In [ ]:
labels_df